# 🤖 Acolhe.AI - Agente de Primeiros Socorros Psicológicos

**⚠️ AVISO IMPORTANTE ⚠️**

Este projeto é um protótipo experimental de um assistente de primeiros socorros psicológicos baseado em Inteligência Artificial (Google Gemini). **ELE NÃO SUBSTITUI AJUDA PROFISSIONAL.** Em momentos de crise grave ou risco para você ou outros, por favor, procure imediatamente um profissional de saúde mental ou ligue para serviços de emergência como o **CVV (Centro de Valorização da Vida) no número 188** no Brasil.

Este bot não é treinado como terapeuta, não pode diagnosticar ou tratar condições de saúde mental. Ele oferece apenas técnicas básicas de manejo para momentos de angústia leve a moderada.

---

## Como Utilizar Este Notebook

O notebook é dividido em blocos de código chamados “células”. Você deve executar cada célula de cima para baixo, clicando no ícone ▶️ (play) ao lado dela.

✅ Etapa 1: Instalar as bibliotecas

Clique no botão ▶️ da primeira célula para instalar os pacotes necessários.

✅ Etapa 2: Inserir a sua API Key

Ao rodar essa célula, o notebook vai pedir que você cole a sua API Key do Google AI Studio.

Ela é necessária para o chatbot funcionar.

Se você não tem ainda:
- Vá para Google AI Studio
- Crie sua chave (grátis).
- Copie e cole no campo solicitado.

⚠️ Importante: Nunca cole sua chave direto no código nem compartilhe ela. Este notebook foi feito para usar a chave de forma segura.

✅ Etapa 3: Início da Conversa

Assim que a API estiver conectada:

Clique no botão ▶️ da segunda célula para iniciar a conversa

---

In [35]:
!pip install -q google-generativeai gradio requests

import google.generativeai as genai
import gradio as gr
import re, threading, time
import requests
import os
import ipywidgets as widgets
from IPython.display import display, HTML

caixa_api_key = widgets.Password(
    placeholder='Cole aqui sua API Key do Google AI Studio',
    description='API Key:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': 'initial'}
)

botao_configurar_api = widgets.Button(
    description='Conectar API',
    button_style='success',
    layout=widgets.Layout(width='18%')
)

status_api = widgets.Output()
linha_api = widgets.HBox([caixa_api_key, botao_configurar_api])

def configurar_api_key(_):
    with status_api:
        status_api.clear_output()
        if not caixa_api_key.value.strip():
            print("⚠️ Por favor, insira uma chave válida.")
            return
        try:
            genai.configure(api_key=caixa_api_key.value.strip())
            print("✅ API conectada com sucesso!")
            print("📌 Agora execute a célula abaixo para iniciar o Acolhe.AI.")
            caixa_api_key.disabled = True
            botao_configurar_api.disabled = True
        except Exception as e:
            print(f"❌ Erro ao configurar a API: {e}")

botao_configurar_api.on_click(configurar_api_key)

display(widgets.VBox([
    widgets.HTML("<h2>🔐 Conecte-se ao Acolhe.AI</h2><p style='font-size:14px;'>Insira sua chave de API para começar.</p>"),
    linha_api,
    status_api
]))


In [39]:
# Configura modelo (já deve ter sido feita a conexão)
modelo_nome = "gemini-2.0-flash"
instrucao_do_agente = """
Você é um assistente de primeiros socorros psicológicos, calmo, empático e objetivo. Seu papel é identificar se o usuário precisa de apoio imediato por uma das seguintes razões:

1. Se a pessoa expressar sofrimento intenso, desesperança, frases como "quero sumir", "acabar com tudo", ou indicar risco — responda APENAS com: [[CRISE]]
2. Se a pessoa demonstrar sintomas de ansiedade intensa, como falta de ar, coração acelerado ou pânico — responda APENAS com: [[RESPIRAR]]
3. Se ela relatar sensação de irrealidade, confusão mental, desatenção, "nada parece real", "estou fora do meu corpo", "estou desligado" — responda APENAS com: [[ANCORAR]]
4. Se a pessoa quiser encerrar — responda APENAS com: [[SAIR]]
5. Caso contrário, responda com frases curtas, empáticas e diretas.

Nunca explique ou misture a tag com texto. Responda SOMENTE com a tag quando identificar uma dessas situações.
"""

model = genai.GenerativeModel(model_name=modelo_nome, system_instruction=instrucao_do_agente)
chat = model.start_chat(history=[])

palavras_chave_emergencia = ["sumir", "acabar com isso", "morrer", "quitar a vida", "não aguento mais", "me matar", "suicídio", "quero me matar"]

# Estilos de chat
style_ia = """
    background-color: #e6f7ff; color: #000; padding: 10px; border-radius: 10px; margin: 5px 0;
    width: fit-content; max-width: 80%; word-wrap: break-word; align-self: flex-start; line-height: 1.4;
"""

style_usuario = """
    background-color: #d1ffd1; color: #000; padding: 10px; border-radius: 10px; margin: 5px 0;
    width: fit-content; max-width: 80%; word-wrap: break-word; align-self: flex-end; margin-left: auto; line-height: 1.4;
"""

chat_scrollable = widgets.Box([chat_box])
chat_scrollable.add_class("chat-scroll")
chat_scrollable.layout = widgets.Layout(
    border='1px solid #ccc',
    padding='10px',
    height='300px',
    overflow_y='auto',
    flex_flow='column',
    display='flex',
    flex_direction='column'
)

main_box.layout = widgets.Layout(
    width='100%',
    max_width='800px',
    margin='0 auto',
    border='1px solid #ccc',
    border_radius='10px',
    padding='20px',
    background_color='#fefefe'
)

def exibir_mensagem(autor, texto):
    texto_html = texto.replace('\n', '<br>')
    texto_html = re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', texto_html)

    estilo = style_usuario if autor == "Você" else style_ia

    msg = widgets.HTML(
        f"<div style='{estilo}'>{texto_html}</div>"
    )

    chat_box.children = list(chat_box.children) + [msg]

thinking_widget = widgets.HTML("⏳ Acolhe.AI está digitando...")

def animar_pensando():
    dots = ["•  ", "• • ", "• • •"]
    i = 0
    while not parar_animacao:
        thinking_widget.value = f"<div style='color:#888;margin-left:10px;'>{'🤖 digitando ' + dots[i % 3]}</div>"
        time.sleep(0.5)
        i += 1

def iniciar_pensando():
    global parar_animacao
    parar_animacao = False
    chat_box.children = list(chat_box.children) + [thinking_widget]
    threading.Thread(target=animar_pensando).start()

def parar_pensando():
    global parar_animacao
    parar_animacao = True
    if thinking_widget in chat_box.children:
        chat_box.children = tuple(c for c in chat_box.children if c != thinking_widget)

def aplicar_tecnica(tecnica):
    if tecnica == 'respirar':
        passos = [
            "Vamos respirar juntos agora...",
            "Inspire lentamente contando até 4...",
            "Segure o ar contando até 4...",
            "Expire suavemente contando até 6...",
            "Muito bem. Continue no seu ritmo por mais um minuto se sentir que ajuda.",
            "Concentre-se apenas no ar entrando e saindo.",
            "Pronto. Como você se sente agora?"
        ]
    elif tecnica == 'ancorar':
        passos = [
            "Vamos tentar a técnica de ancoragem 5-4-3-2-1.",
            "Diga 5 coisas que você vê ao seu redor...",
            "Agora, 4 coisas que pode tocar...",
            "3 sons que consegue ouvir...",
            "2 cheiros que percebe ou lembra...",
            "1 sabor que sente ou pode imaginar...",
            "Pronto. Isso ajuda a focar nos seus sentidos e no ambiente ao redor.",
            "Como você se sente agora?"
        ]
    else:
        return

    for passo in passos:
        exibir_mensagem("Acolhe.AI", passo)
        time.sleep(4)

def exibir_resposta_em_partes(texto, pausa=2.5):
    partes = re.split(r'(?<=[.!?])\s+', texto.strip())
    for parte in partes:
        exibir_mensagem("Acolhe.AI", parte)
        time.sleep(pausa)

def processar_mensagem(_):
    mensagem = input_box.value.strip()
    if not mensagem:
        return
    input_box.value = ""
    exibir_mensagem("Você", mensagem)

    mensagem_lower = mensagem.lower()

    if any(p in mensagem_lower for p in palavras_chave_emergencia):
        resposta_crise = [
            "⚠️ Suas palavras indicam um sofrimento muito intenso.",
            "Eu sou apenas um assistente de primeiros socorros emocionais e **NÃO** consigo ajudar em situações graves.",
            "Por favor, procure ajuda especializada agora mesmo.",
            "Ligue para o **CVV (188)** ou para serviços de emergência.",
            "Você não está sozinho(a). Fale com alguém de confiança."
        ]
        for msg in resposta_crise:
            exibir_mensagem("Acolhe.AI", msg)
            time.sleep(2.5)
        return

    # 🎯 Gatilhos para técnicas manuais
    if mensagem_lower in ['1', '4'] or "respirar" in mensagem_lower or "ansioso" in mensagem_lower:
        aplicar_tecnica('respirar')
        return
    if mensagem_lower == '2' or 'ancora' in mensagem_lower or 'desorientado' in mensagem_lower:
        aplicar_tecnica('ancorar')
        return


    iniciar_pensando()
    try:
        resposta = chat.send_message(mensagem)
        texto = resposta.text.strip()
        parar_pensando()

        if texto.startswith('[[CRISE]]'):
            resposta_crise = [
                "⚠️ Suas palavras indicam um sofrimento muito intenso.",
                "Eu sou apenas um assistente de primeiros socorros emocionais e **NÃO** consigo ajudar em situações graves.",
                "Por favor, procure ajuda especializada agora mesmo.",
                "Ligue para o **CVV (188)** ou para serviços de emergência.",
                "Você não está sozinho(a). Fale com alguém de confiança."
            ]
            for msg in resposta_crise:
                exibir_mensagem("Acolhe.AI", msg)
                time.sleep(2.5)
        elif texto.startswith('[[RESPIRAR]]'):
            aplicar_tecnica('respirar')
        elif texto.startswith('[[ANCORAR]]'):
            aplicar_tecnica('ancorar')
        elif texto.startswith('[[SAIR]]'):
            despedida = [
                "Tudo bem, vamos encerrar por aqui. 🌱",
                "Lembre-se: você pode voltar quando quiser.",
                "Desejo calma e clareza para o seu momento.",
                "Se precisar de apoio humano, o CVV está disponível pelo número 188.",
                "Até breve!"
            ]
            for msg in despedida:
                exibir_mensagem("Acolhe.AI", msg)
                time.sleep(2.5)
            return

        else:
            texto_limpo = re.sub(r'\[\[.*?\]\]', '', texto).strip()
            exibir_resposta_em_partes(texto_limpo)

    except Exception as e:
        parar_pensando()
        exibir_mensagem("Acolhe.AI", f"❌ Ocorreu um erro: {e}")

send_button.on_click(processar_mensagem)
input_box.on_submit(processar_mensagem)

# UI completa
titulo = widgets.HTML("<h1 style='text-align:center; color:#3498db'>🤖 Acolhe.AI</h1>")
descricao = widgets.HTML("""
<p style='text-align: center; color: #555; font-size: 14px;'>Acolhe.AI, seu assistente de primeiros socorros psicológicos.</p>
<p style='font-size: 13px; color:#666; padding:10px; border:1px solid #ccc; border-radius:10px; background-color:#f9f9f9;'>
⚠️ AVISO IMPORTANTE ⚠️<br>
Este projeto é um protótipo experimental. <b>ELE NÃO SUBSTITUI AJUDA PROFISSIONAL.</b><br>
Se estiver em crise grave, ligue para o <b>CVV (188)</b> ou procure ajuda especializada.
</p>
""")

entrada = widgets.HBox([input_box, send_button])
main_box = widgets.VBox([titulo, descricao, chat_scrollable, entrada])
main_box.layout = widgets.Layout(width='70%', margin='20px auto')

display(main_box)

# 🧼 Limpa o histórico do chat e a interface
chat = model.start_chat(history=[])  # reseta o histórico do modelo
chat_box.children = []  # limpa as mensagens já exibidas

# Mensagens iniciais
mensagens_iniciais = [
    "Estou aqui para você. Para começar, como você está se sentindo agora?",
    "**Por favor, digite o número da opção que melhor descreve seu estado, ou descreva com suas palavras:**\n1. Muito ansioso(a) ou em pânico\n2. Desorientado(a) ou fora do meu corpo\n3. Angustiado(a) ou sobrecarregado(a)\n4. Preciso de ajuda para acalmar a respiração\n5. Outro (vou descrever com minhas palavras)\n\nOu digite 'sair' para encerrar."
]

for msg in mensagens_iniciais:
    exibir_mensagem("Acolhe.AI", msg.strip())

